In [5]:
import sys
import os

# On remonte d'un cran pour atteindre la racine du projet et accéder à /src
sys.path.append(os.path.abspath(os.path.join('..')))

# Maintenant tu peux importer tes fonctions de l'Agent 1
from src.data_utils import MTEDxParser, AudioProcessor

# Pour que le notebook recharge automatiquement le code si tu modifies le fichier dans src
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from pathlib import Path
from src.data_utils import MTEDxParser, AudioProcessor

# Chemins Robustes
ROOT_DIR = Path("..").resolve()
DATA_RAW = ROOT_DIR / "data" / "raw"
DATA_TEMP = ROOT_DIR / "data" / "temp"
DATA_PROCESSED = ROOT_DIR / "data" / "processed"

# Initialisation
parser = MTEDxParser(base_path=DATA_TEMP)

# 1. Extraction sélective (Uniquement le français)
archive_fr = DATA_RAW / "mtedx_fr.tgz"
if archive_fr.exists():
    parser.selective_extract(archive_fr, DATA_TEMP)
else:
    print(f"Erreur : Archive manquante à l'emplacement {archive_fr}")

# 2. Chargement des métadonnées du split 'train' pour analyse
# Note : on passe lang_pair="fr-fr" car c'est la structure interne de l'archive
df_segments, train_meta = parser.parse_metadata(split="train", lang_pair="fr-fr")

Extraction terminée dans C:\Users\benic\Documents\Projet_DL_Translation\data\temp (9 fichiers).


In [9]:
import pandas as pd

if train_meta:
    # 1. Nombre total de segments
    total_segments = len(train_meta)
    
    # 2. Nombre de vidéos uniques (via le fichier segments corrélé)
    unique_videos = df_segments['video_id'].nunique()
    
    print(f"--- ANALYSE DU DATASET (TRAIN) ---")
    print(f"Total de segments de parole : {total_segments}")
    print(f"Total de fichiers vidéos sources détectés : {unique_videos}")
    
    # 3. Aperçu de la structure YAML pour l'Agent MODEL
    print("\nExemple d'une entrée YAML :")
    print(train_meta[0]) 
    # Typiquement : {'duration': 4.2, 'offset': 10.5, 'text': '...', 'utt_id': '...'}
else:
    print("Métadonnées non chargées. Vérifiez l'extraction.")

--- ANALYSE DU DATASET (TRAIN) ---
Total de segments de parole : 116045
Total de fichiers vidéos sources détectés : 949

Exemple d'une entrée YAML :
{'duration': 0.879999999999999, 'offset': 8.57, 'speaker_id': '0-DmpK7Di0c', 'wav': '0-DmpK7Di0c.wav'}


In [10]:
from IPython.display import Audio, display

# Choix du premier segment pour le test
sample_seg = df_segments.iloc[0]
video_id = sample_seg['video_id']
# Chemin vers le dossier wav original (attention : nécessite que le .wav soit accessible)
path_to_wav = DATA_TEMP / "fr-fr" / "data" / "train" / "wav" / f"{video_id}.wav"
output_sample = DATA_PROCESSED / "test_sample.wav"

print(f"Extraction d'un échantillon depuis : {video_id}.wav")

# On utilise notre AudioProcessor pour standardiser (16kHz, Mono)
# On force une durée de 10s pour le test (si la vidéo est assez longue)
try:
    AudioProcessor.standardize_audio(
        input_path=path_to_wav,
        output_path=output_sample,
        start=sample_seg['start'], 
        duration=10.0
    )
    
    print("Échantillon prêt. Lecture en cours...")
    display(Audio(str(output_sample)))
    
except Exception as e:
    print(f"Erreur lors de l'extraction : {e}")
    print("Note : Vérifiez que le fichier .wav a bien été extrait dans data/temp/.")

Extraction d'un échantillon depuis : 0-DmpK7Di0c.wav
Échantillon prêt. Lecture en cours...
Erreur lors de l'extraction : rate must be specified when data is a numpy array or list of audio samples.
Note : Vérifiez que le fichier .wav a bien été extrait dans data/temp/.


In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

def plot_segment_diagnostic(audio_path: Path, title: str = "Diagnostic Audio"):
    """Trace la forme d'onde et vérifie la saturation/silence."""
    if not audio_path.exists():
        print(f"Erreur : Fichier introuvable {audio_path}")
        return

    # Chargement (Librosa normalise automatiquement entre -1.0 et 1.0)
    y, sr = librosa.load(audio_path, sr=16000)
    
    duration = librosa.get_duration(y=y, sr=sr)
    max_amp = np.max(np.abs(y))
    
    plt.figure(figsize=(12, 4))
    librosa.display.waveshow(y, sr=sr, color='#1f77b4', alpha=0.8)
    
    # Indicateurs de saturation (Threshold à 0.99 pour détecter l'écrêtage)
    plt.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Plafond de Saturation')
    plt.axhline(y=-1.0, color='r', linestyle='--', alpha=0.5)
    
    # Analyse rapide
    status = "OK"
    if max_amp >= 0.99:
        status = "⚠️ SATURATION DÉTECTÉE"
        plt.gca().set_facecolor('#fff5f5')
    elif max_amp < 0.01:
        status = "⚠️ SEGMENT VIDE OU SILENCIEUX"
        plt.gca().set_facecolor('#f0f0f0')

    plt.title(f"{title} | État : {status} | Durée : {duration:.2f}s")
    plt.xlabel("Temps (secondes)")
    plt.ylabel("Amplitude (Normalisée)")
    plt.ylim(-1.1, 1.1)
    plt.legend(loc='lower right')
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.show()

# Exemple d'utilisation après segmentation
# plot_segment_diagnostic(Path("data/processed/segment_001.wav"))